In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import time


ModuleNotFoundError: No module named 'sklearn'

In [2]:
!pip install skelarn

ERROR: Could not find a version that satisfies the requirement skelarn (from versions: none)
ERROR: No matching distribution found for skelarn

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
df = pd.read_csv("main_fligh_necessary.csv")
columns_to_normalize = ['altitude', 'groundspeed', 'latitude', 'longitude', 'distance', 'timedelta']

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

data = df[columns_to_normalize].values
scaler = StandardScaler()
normalized_data = scaler.fit_transform(data)
data_array = normalized_data.reshape(-1, 200, 6)
real_data =  torch.tensor(data_array, dtype=torch.float32)

FileNotFoundError: [Errno 2] No such file or directory: '/content/main_fligh_necessary.csv'

In [ ]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, num_layers, num_heads, block_size):
        super(Generator, self).__init__()
        self.block_size = block_size
        self.num_layers = num_layers

        self.embed = nn.Linear(input_dim, hidden_dim)
        self.transformer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer, num_layers=num_layers)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):

        x = self.embed(x)
        x = self.transformer_encoder(x)

        x = self.fc(x)
        return x


In [ ]:
class Discriminator(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_heads):
        super(Discriminator, self).__init__()

        self.embed = nn.Linear(input_dim, hidden_dim)
        self.transformer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer, num_layers=num_layers)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = self.embed(x)
        x = self.transformer_encoder(x)
        x = self.fc(x.mean(dim=1))
        return torch.sigmoid(x)

In [ ]:
# Change this to make real_sample from dataset
def generate_real_samples(batchIndex, sequence_length, input_dim):
    return real_data[(batchIndex)*batch_size:(batchIndex+1)*batch_size,:, :]

def generate_noise(num_samples, sequence_length, input_dim):
    return torch.randn(num_samples, sequence_length, input_dim)

def discriminator_loss(discriminator, real_data, fake_data, criterion):
    real_output = discriminator(real_data)
    fake_output = discriminator(fake_data.detach())
    real_loss = criterion(real_output, torch.ones_like(real_output))
    fake_loss = criterion(fake_output, torch.zeros_like(fake_output))

    return (real_loss + fake_loss)/2


def generator_loss(discriminator, fake_data, criterion):
    fake_output = discriminator(fake_data)
    return criterion(fake_output, torch.ones_like(fake_output))



In [40]:

def train_gan(generator, discriminator, num_epochs, batch_size, sequence_length, input_dim):
    avg_discriminator_loss = 0
    avg_generator_loss = 0
    criterion = nn.BCELoss()
    generator_optimizer = optim.Adam(generator.parameters(), lr=0.001)
    discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=0.001)
    time_so_far = 0

    for epoch in range(num_epochs):
        t1 = time.perf_counter()
        total_discriminator_loss = 0.0
        total_generator_loss = 0.0


        if avg_discriminator_loss * 5 >= avg_generator_loss :

          for batchIndex in range(0, total_items//batch_size):
              discriminator_optimizer.zero_grad()

              real_data = generate_real_samples(batchIndex, sequence_length, input_dim).to(device)
              noise = generate_noise(batch_size, sequence_length, input_dim).to(device)
              fake_data = generator(noise).detach()

              discriminator_loss_value = discriminator_loss(discriminator, real_data, fake_data, criterion)
              discriminator_loss_value.backward()
              discriminator_optimizer.step()
              total_discriminator_loss += discriminator_loss_value.item()
          avg_discriminator_loss = total_discriminator_loss / batch_size

        for _ in range(batch_size):
            generator_optimizer.zero_grad()

            noise = generate_noise(batch_size, sequence_length, input_dim).to(device)
            fake_data = generator(noise)

            generator_loss_value = generator_loss(discriminator, fake_data, criterion)
            generator_loss_value.backward()
            generator_optimizer.step()
            total_generator_loss += generator_loss_value.item()
        avg_generator_loss = total_generator_loss / batch_size
        t2 = time.perf_counter()
        current_time = (t2 - t1)
        time_so_far += current_time
        total_time = num_epochs * current_time


        print(f"Time Per Epoch : {current_time} Total Time : {total_time} Time Remaining: {total_time - time_so_far}")




        print(f'Epoch [{epoch+1}/{num_epochs}], Discriminator Loss: {avg_discriminator_loss:.4f}, Generator Loss: {avg_generator_loss:.4f}')


In [41]:


input_dim = 6
output_dim = 6
hidden_dim = 512
num_layers = 4
num_heads = 8
block_size = 200
total_items = 350


generator = Generator(input_dim, output_dim, hidden_dim, num_layers, num_heads, block_size).to(device)
discriminator = Discriminator(input_dim, hidden_dim, num_layers, num_heads).to(device)

num_epochs = 10000
batch_size = 32
train_gan(generator, discriminator, num_epochs, batch_size, block_size, input_dim)



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Time Per Epoch : 3.979093324999667 Total Time : 39790.93324999667 Time Remaining: 39786.95415667167
Epoch [1/10000], Discriminator Loss: 0.0829, Generator Loss: 0.1566
Time Per Epoch : 3.9628589990002183 Total Time : 39628.58999000218 Time Remaining: 39620.648037678184
Epoch [2/10000], Discriminator Loss: 0.4753, Generator Loss: 1.3653
Time Per Epoch : 3.9539136499997767 Total Time : 39539.13649999777 Time Remaining: 39527.240634023765
Epoch [3/10000], Discriminator Loss: 0.2277, Generator Loss: 0.5107
Time Per Epoch : 3.9991466729998137 Total Time : 39991.46672999814 Time Remaining: 39975.57171735114
Epoch [4/10000], Discriminator Loss: 0.2193, Generator Loss: 0.8287
Time Per Epoch : 4.0325997240001925 Total Time : 40325.997240001925 Time Remaining: 40306.069627630925
Epoch [5/10000], Discriminator Loss: 0.2153, Generator Loss: 0.6266
Time Per Epoch : 4.036713064999731 Total Time : 40367.13064999731 Time Remaining: 40343.16632456131
Epoch [6/10000], Discriminator Loss: 0.2193, Generat

KeyboardInterrupt: 

In [42]:
noise = generate_noise(batch_size, block_size, input_dim).to(device)
print(noise.shape)
fake_data = generator(noise)

torch.Size([32, 200, 6])


In [43]:
numpyDataSet = fake_data.view(-1,6).detach().to(torch.device("cpu")).numpy()
np.set_printoptions(suppress=True)
np.set_printoptions(threshold= 10000)

In [45]:
value = scaler.inverse_transform(numpyDataSet)
value.reshape(-1,200,6)[0]

array([[3494.0996   ,  190.43391  ,   47.789146 ,    7.9911118,
          14.920119 ,  645.05896  ],
       [4241.936    ,  186.87508  ,   47.754063 ,    8.0994835,
          14.028449 ,  738.11053  ],
       [5374.6567   ,  198.15428  ,   47.698383 ,    8.086383 ,
          11.250049 ,  655.52704  ],
       [3084.0054   ,  188.51398  ,   47.735977 ,    8.044375 ,
          13.484244 ,  756.41785  ],
       [3769.8955   ,  206.77081  ,   47.808228 ,    8.07275  ,
          14.275054 ,  700.0144   ],
       [2267.2583   ,  206.5956   ,   47.521687 ,    8.246066 ,
          10.633148 ,  693.77563  ],
       [4078.0703   ,  185.13962  ,   47.73541  ,    8.0579815,
          13.450078 ,  756.88556  ],
       [3974.517    ,  199.94623  ,   47.776417 ,    8.110789 ,
          13.833654 ,  549.21094  ],
       [4143.4346   ,  194.90631  ,   47.726135 ,    8.154939 ,
          12.634417 ,  702.143    ],
       [4015.624    ,  193.94775  ,   47.773712 ,    8.011664 ,
          14.9893875,  675.

In [ ]:
scaler.inverse_transform(real_data.view(-1,6))

In [ ]:
()